# Matrix Factorization using Gradient Descent

In this excercise, you are required to implement matrix factorization method, specifically [Non-Negative Matrix Factorization (NMF)](https://en.wikipedia.org/wiki/Non-negative_matrix_factorization), using gradient descent. You have to apply the matrix factorization to solve topic modeling. 

(Please refer to the tutorial on basics of topic modeling, LSI with SVD (Tutorial Set 4), for details on LSI)

## Applying NMF to solve Topic Modeling
Given a term document matrix $V$, NMF factorizes it into two matrix $W$ and $H$ with the property that all three documents have no negative elements.
<img src="content/NMF.png" alt="Non-negative matrix factorization" style="width: 80%">

In Non-negative Matrix Factorization, a document-term matrix is approximately factorized into term-feature and feature-document matrices.

$V = WH$ Matrix multiplication can be implemented as computing the column vectors of $V$ as linear combinations of the basis vectors (column vectors) in $W$ (or the topics discovered from the documents) using coefficients supplied by columns of $H$ (or the membership weights for the topics in each document). That is, each column of V can be computed as follows:
$$ v_i = W h_i$$

In what follows, we will first see an example of applying NMF by using [SKLearn NMF API](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) for the task of topic modeling. Later you will be required to implement NMF using gradient descent.

### Scikit-Learn implementation of NMF for topic modeling
Given a set of multivariate  $n$-dimensional data vectors, they are put into an  $n\times m$  matrix  $V$  as its columns, where  $m$  is the number of examples in the data set. This matrix  $V$  is approximately factorized into an  $n \times t$  matrix  $W$  and an  $t \times m$  matrix  $H$ , where  $t$  is generally less than  $n$  or  $m$ . Hence, this results in a compression of the original data matrix.

In terms of topic modeling, the input document-term matrix  $V$  is factorized into a  $n \times t$  document-topic matrix and a  $t \times m$  topic-term matrix, where  $t$  is the number of topics produced. Similar to tutorial 4, we will be using 20 NewsFetch dataset for the task.

Name: Avani Gupta <br>
Roll: 2019121004

#### Imports

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

%matplotlib inline

#### Setup data

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)


#### Compute document features

In [3]:
vectorizer_tfidf = TfidfVectorizer(stop_words='english')
vectors_tfidf = vectorizer_tfidf.fit_transform(newsgroups_train.data).todense() # (documents, vocab)
vectors_tfidf.shape

(2034, 26576)

#### Compute NMF using Scikit Learn library

We will also write a function to display top 8 words for each topic.

In [4]:
num_top_words=8
vocab = np.array(vectorizer_tfidf.get_feature_names())

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [5]:
from sklearn import decomposition

d = 5 # num topics
clf = decomposition.NMF(n_components=d, random_state=1)

In [6]:
W1 = clf.fit_transform(vectors_tfidf)
H1 = clf.components_

In [7]:
show_topics(H1)

['people don think just like objective say morality',
 'graphics thanks files image file program windows know',
 'space nasa launch shuttle orbit moon lunar earth',
 'ico bobbe tek beauchaine bronx manhattan sank queens',
 'god jesus bible believe christian atheism does belief']

# Excercise

## NMF using SGD

In stochastic gradient descent (SGD), we evaluate our loss function on just a sample of our data (sometimes called a mini-batch). We would get different loss values on different samples of the data, so this is why it is stochastic. It turns out that this is still an effective way to optimize, and it's much more efficient!

### Applying SGD to NMF

Goal: Decompose $V\;(m \times n)$ into
$$ V \approx HW$$
where $W\;(m \times d)$ and $H\;(d \times n)$, $W,\;H\; \geq \;0$, and we've minimized the Frobenius norm of $V-WH$. The objective function can therefore be written as the following:
$$
\min_{H \geq 0, W \geq 0} F(H,W) = \frac{1}{2} ||V - HW||^{2} + \frac{\lambda}{2} \left( ||H||^2 + ||W||^2 \right)
$$

### Implementation of NMF using SGD (Excercise)
__Approach:__ Given the objective function above, pick random positive $W$ & $H$, and then use SGD to optimize. 

(Note that the objective function is non-convex in nature, and is convex only if we consider $H$ and $W$ separately. You can directly write the gradient descent rule for the objective function presented above)



In [8]:
''' WRITE YOUR CODE BELOW '''
mu = 1e-6
lambdaa=1e3
lr=1e-2
rows, cols = vectors_tfidf.shape
def updateGrads(M, W, H,lr):
    R = W@H-M
    dW = R@H.T
    dW += np.where(W>=mu,0, np.min(W - mu, 0))*lambdaa
    dH = W.T@R
    dH += np.where(H>=mu,0, np.min(H - mu, 0))*lambdaa 
    W -= lr*dW
    H -= lr*dH
    

def printvals(M,W,H):
    negW = W<0
    negH = H<0
    negWnum = negW.sum()
    negHnum = negH.sum()
    norm = np.linalg.norm(M-W@H)
    print("norm:",norm, "min W:",W.min(), "min H:", H.min(), "num of negative W:",negWnum ,"num of negative H:",negHnum)
    
W = np.random.normal(scale=0.01, size=(rows,d))
W = np.abs(W)
H = np.random.normal(scale=0.01, size=(d,cols))
H = np.abs(H) 
print("initial")
printvals(vectors_tfidf, W, H)
updateGrads(vectors_tfidf,W,H,lr)
print("after first update")
printvals(vectors_tfidf, W, H)


for i in range(1000): 
    updateGrads(vectors_tfidf,W,H,lr)
    if i % 50 == 0:
        print("iter ",i)
        printvals(vectors_tfidf, W, H)

print("topics")        
show_topics(H)

initial
norm: 44.4264794074542 min W: 6.240378201888802e-07 min H: 5.099848891127229e-08 num of negative W: 0 num of negative H: 0
after first update
norm: 44.41859490841542 min W: -0.000880985585715645 min H: -8.833662674790848e-05 num of negative W: 139 num of negative H: 290
iter  0
norm: 44.412916654420606 min W: -0.0007093268082925707 min H: -7.769420423041111e-05 num of negative W: 111 num of negative H: 262
iter  50
norm: 44.24419771663191 min W: -0.00013440935286741776 min H: -0.00013246610552814802 num of negative W: 36 num of negative H: 2966
iter  100
norm: 44.14123536919818 min W: -0.00018773958445390292 min H: -0.0001782590480765164 num of negative W: 58 num of negative H: 6898
iter  150
norm: 44.08253742014953 min W: -0.0015134271088304734 min H: -0.00040749337037090873 num of negative W: 123 num of negative H: 8738
iter  200
norm: 43.988354915062494 min W: -0.0034952934096576763 min H: -0.0008306918047459974 num of negative W: 133 num of negative H: 9339
iter  250
norm: 

['think don people just objective like morality moral',
 'ico bobbe tek beauchaine bronx manhattan sank queens',
 'god jesus bible believe christian atheism does belief',
 'thanks graphics files image file program windows know',
 'space nasa launch shuttle orbit moon lunar earth']